In [1]:
from preamble import *
from preamble.dynesty.pooled_lw_rot import *

In [2]:
from functools import partial
from os.path import isfile

@partial(jax.jit, static_argnums=(1,2))
def ln_like(θ, without=-1, without_l=-1):
#     return sum(ln_like_chi2_2dof(m, o) for m, o in zip(model(θ), chunk_PS))
    return ln_like_chi2_2dof(
        chunk_model(ν, θ, without=without, without_l=without_l), 
        PS)
reset = False

if (not isfile(checkpoint)) or reset:
    sampler = dynesty.DynamicNestedSampler(ln_like, prior_transform, Ndims, reflective=[0])
    sampler.run_nested()
    summary = summarise(sampler)
    np.save(checkpoint, {'summary': summary})
else:
    d = np.load(checkpoint, allow_pickle=True)[()]
    summary = d['summary']
    
from os import makedirs
from os.path import basename
figdir = f"figures/{basename(checkpoint).replace('.npy', '')}"
makedirs(figdir, exist_ok=True)

In [3]:
from scipy.optimize import minimize

# get ML solution

θ0 = summary['new_samples'][-1]
f = lambda *a: -ln_like(*a)
j0 = minimize(f, θ0, method='L-BFGS-B',
              options=dict(maxiter=30000))
θ = j0['x']
j0 = minimize(f, θ, method='Nelder-Mead',
              options=dict(maxiter=30000, ))
θ = j0['x']
j0 = minimize(f, θ, method='Nelder-Mead',
              options=dict(maxiter=30000, ))
θ = j0['x']
j0 = minimize(f, θ, method='TNC')
θ = j0['x']
q0 = j0['fun']
j0

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


 message: Converged (|f_n-f_(n-1)| ~= 0)
 success: True
  status: 1
     fun: 927.542724609375
       x: [ 1.436e+00  4.804e-02 ...  8.845e-01  2.115e-01]
     nit: 1
     jac: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
    nfev: 722

In [4]:
from scipy.stats.distributions import chi2
for i, _ in enumerate(nu_guess):
    f = lambda *a: -ln_like(*a, without=i)
    j = minimize(f, θ, method='TNC')
    q = j['fun']
    if np.isnan(q):
        q = -ln_like(θ, without=i)

    # q = 1/2 χ^2
    # Likelihood ratio is just exp(Δq)
    
    Λ = 2 * (q - q0)
    LR = np.exp(q0 - q)
    
    # χ2-distributed, with 2 degrees of freedom
    # (ν and A)
    p = chi2.sf(Λ, 2)

    print(f"{Λ:.3f} | {p:.3e} | {q:.2f}")

69.778 | 7.045e-16 | 962.43
28.152 | 7.705e-07 | 941.62
38.650 | 4.048e-09 | 946.87
2.602 | 2.723e-01 | 928.84
17.942 | 1.271e-04 | 936.51
14.586 | 6.804e-04 | 934.84
35.977 | 1.541e-08 | 945.53
45.037 | 1.661e-10 | 950.06
37.975 | 5.673e-09 | 946.53
10.398 | 5.523e-03 | 932.74
1.072 | 5.851e-01 | 928.08
28.583 | 6.213e-07 | 941.83
35.264 | 2.200e-08 | 945.17
106.270 | 8.390e-24 | 980.68
24.919 | 3.881e-06 | 940.00


In [5]:
f = lambda *a: -ln_like(*a, without_l=2)
j = minimize(f, θ, method='TNC')
q = j['fun']
if np.isnan(q):
    q = -ln_like(θ, without_l=2)

# q = 1/2 χ^2
# Likelihood ratio is just exp(Δq)

Λ = 2 * (q - q0)
LR = np.exp(q0 - q)

# χ2-distributed, with 2*5 + 1 = 11 degrees of freedom
# (ν and A) for 5 modes, and quadrupole-mode rotation rate.
p = chi2.sf(Λ, 11)

print(q)
print(f"{Λ:.3f} | {p:.3e}")

1028.203369140625
201.321 | 3.956e-37
